# Evaluation Walkthrough

Automate process to one click deliver

In [1]:
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: datascience-covid19'

# Update all data

In [2]:
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( 'data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))
    
def get_current_data_germany():

    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( 'data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))    



if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   0314c8b..82c07de  web-data   -> origin/web-data\n'
out : b'Already up-to-date.\n'
Error : b''
out : b'Already up-to-date.\n'


# Process Pipeline

In [3]:
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    
    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({
        'date':time_idx})
    df_input_large= pd_raw['Country/Region'].unique()
    for each in df_input_large:
        df_plot[each] =np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))
    df = df_plot.drop('date', axis=1)
    
    #Merging the data set over COUNTRY for CODE column
    world_raw =  pd.DataFrame({"COUNTRY" : df_input_large, "Confirm cases" :df.iloc[-1]})
    
    pd_data_base=pd_raw.rename(columns={'Country/Region':'COUNTRY',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','COUNTRY']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
    
    pd_relational_model.to_csv('data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    
if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 63042


# Filter and Doubling rate calculation

In [4]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function such that the data struture is kept '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame'''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','COUNTRY',filter_on]].groupby(['state','COUNTRY']).apply(savgol_filter)#.reset_index()

   
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame '''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','COUNTRY']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()


    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['COUNTRY']=='Germany'].tail())


the test slope is: [2.]
            date state  COUNTRY  confirmed  confirmed_filtered  confirmed_DR  \
34360 2020-09-10    no  Germany   258149.0            258018.2    160.722431   
34361 2020-09-11    no  Germany   259735.0            259374.2    156.332930   
34362 2020-09-12    no  Germany   260817.0            260732.0    194.577961   
34363 2020-09-13    no  Germany   261737.0            261946.8    260.502498   
34364 2020-09-14    no  Germany   263222.0            263161.6    217.817325   

       confirmed_filtered_DR  
34360             168.789051  
34361             184.661656  
34362             191.152480  
34363             202.662158  
34364             215.629569  


  # Visual Board

In [ ]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State


import plotly.graph_objects as go
from plotly import tools

import os
print(os.getcwd())
df_input_large = pd.read_csv('data/processed/COVID_final_set.csv',sep=';')

fig=go.Figure()

app=dash.Dash()
app.layout=html.Div([
    dcc.Markdown('''Applied data science on COVID-19 data
    
    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard. '''),
    
    dcc.Markdown('''
    ## Multi-Select Country for visualization '''),
    
    dcc.Dropdown(
        id='country_dropdown',
        options=[ {'label': each,'value':each} for each in df_input_large['COUNTRY'].unique()],
        value=['US', 'Germany','India'], # which are pre-selected
        multi= True,
        
    ),
    
    dcc.Dropdown(
        id='doubling_time',
        options=[
              {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
              {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
              {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
              {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'}
            ],
        value='confirmed',
        multi=False
        ),
    
    dcc.Graph(figure=fig,id='main_window_slope')
    
])


@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_dropdown', 'value'),
    Input('doubling_time', 'value')])

def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['COUNTRY']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                height=600,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        ) 
    }

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

/home/tk_conda/Documents/Shoriya/DataScience/datascience-covid19
Dash is running on http://127.0.0.1:8050/



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
